<a href="https://colab.research.google.com/github/Fabio-RibeiroB/BBK-Applied-ML/blob/main/BBK_AML_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AML Coursework - Cyberattack Classifier


## Connect Google Drive and Import Necessary Libraries

In [125]:
from pandas import read_csv
import numpy
from pandas import set_option
from matplotlib import pyplot
from pandas.plotting import scatter_matrix 

In [121]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
# Training and Test sets
root ='/content/drive/MyDrive/AML Data/'
train_csv = root +  'train_imperson_without4n7_balanced_data.csv'
test_csv = root +  'test_imperson_without4n7_balanced_data.csv'

## Loading the Data

In [119]:
# Get the names of all features from txt file
features_txt = open(root+"features.txt", "r")
features = []
feature = features_txt.readline()

while feature != "":
  feature = feature.rstrip("\n")
  if feature not in features:
    features.append(feature)
  else:
    features.append(feature + '2') # There is a duplicate name, add 2 to the end
  feature = features_txt.readline()

features_txt.close()

# remove features 4 and 7 as required by specification as they provide temporal 
# information which may cause unfair prediction
features.remove('frame.time_epoch')
features.remove('frame.time_relative')


# print(features[-1]) # target variable 'class'
# print(len(features)) # 152 input variables + 1 target

# Put into pd dataframe, remove first row which is just a variable number
train = read_csv(train_csv, names=features)
train = train.drop([0])
print(train.shape) # (97044, 155)

# Class distribution - check if balanced
class_counts=train.groupby('class').size()
print(class_counts) # Balanced data

# Desribe
train.describe()

(97044, 153)
class
0    48522
1    48522
dtype: int64


,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.reserved,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.mactime,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,...,wlan_mgt.fixed.fragment,wlan_mgt.fixed.sequence,wlan_mgt.tagged.all,wlan_mgt.ssid,wlan_mgt.ds.current_channel,wlan_mgt.tim.dtim_count,wlan_mgt.tim.dtim_period,wlan_mgt.tim.bmapctl.multicast,wlan_mgt.tim.bmapctl.offset,wlan_mgt.country_info.environment,wlan_mgt.rsn.version,wlan_mgt.rsn.gcs.type,wlan_mgt.rsn.pcs.count,wlan_mgt.rsn.akms.count,wlan_mgt.rsn.akms.type,wlan_mgt.rsn.capabilities.preauth,wlan_mgt.rsn.capabilities.no_pairwise,wlan_mgt.rsn.capabilities.ptksa_replay_counter,wlan_mgt.rsn.capabilities.gtksa_replay_counter,wlan_mgt.rsn.capabilities.mfpr,wlan_mgt.rsn.capabilities.mfpc,wlan_mgt.rsn.capabilities.peerkey,wlan_mgt.tcprep.trsmt_pow,wlan_mgt.tcprep.link_mrg,wlan.wep.iv,wlan.wep.key,wlan.wep.icv,wlan.tkip.extiv,wlan.ccmp.extiv,wlan.qos.tid,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class
count,97044.0,97044.0,97044.0,9.704e+04,9.704e+04,97044.000,97044.000,97044.0,97044.0,97044.0,97044.0,97044.000,97044.000,97044.000,97044.0,97044.000,97044.0,97044.000,97044.0,97044.0,97044.0,97044.0,97044.0,97044.000,97044.0,97044.0,97044.000,97044.0,97044.0,97044.0,97044.0,97044.0,97044.0,97044.0,97044.0,97044.000,97044.0,97044.0,97044.0,97044.0,...,97044.0,9.704e+04,97044.000,9.704e+04,97044.000,9.704e+04,9.704e+04,9.704e+04,97044.0,9.704e+04,9.704e+04,9.704e+04,9.704e+04,9.704e+04,97044.000,97044.0,97044.0,9.704e+04,97044.0,97044.0,97044.0,97044.0,97044.000,97044.0,97044.000,97044.000,97044.000,97044.000,9.704e+04,97044.000,97044.000,97044.0,97044.0,97044.0,97044.0,97044.0,97044.0,97044.0,97044.000,97044.0
mean,0.0,0.0,0.0,6.252e-03,6.252e-03,0.194,0.194,0.0,0.0,0.0,0.0,1.000,1.000,1.000,1.0,1.000,0.0,1.000,0.0,0.0,0.0,0.0,0.0,1.000,0.0,0.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.714,0.0,0.0,0.0,0.0,...,0.0,6.494e-06,0.090,3.359e-04,0.037,2.127e-05,1.823e-04,6.183e-04,0.0,4.944e-04,3.317e-07,1.690e-04,6.422e-07,3.403e-07,0.022,0.0,0.0,1.030e-05,0.0,0.0,0.0,0.0,0.500,1.0,0.273,0.004,0.261,0.006,3.463e-04,0.028,0.028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.178,0.5
std,0.0,0.0,0.0,1.554e-02,1.554e-02,0.354,0.354,0.0,0.0,0.0,0.0,0.015,0.015,0.015,0.0,0.015,0.0,0.015,0.0,0.0,0.0,0.0,0.0,0.015,0.0,0.0,0.015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.110,0.0,0.0,0.0,0.0,...,0.0,9.109e-04,0.286,1.152e-02,0.126,3.109e-04,9.980e-04,2.486e-02,0.0,1.273e-02,2.236e-06,1.139e-03,4.367e-06,2.294e-06,0.145,0.0,0.0,3.210e-03,0.0,0.0,0.0,0.0,0.001,0.0,0.260,0.036,0.262,0.060,1.331e-02,0.063,0.063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360,0.5
min,0.0,0.0,0.0,2.860e-06,2.860e-06,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.000,1.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.000e+00,0.000,0.000e+00,0.000,0.000e+00,0.000e+00,0.000e+00,0.0,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000,0.0,0.0,0.000e+00,0.0,0.0,0.0,0.0,0.500,1.0,0.000,0.000,0.000,0.000,0.000e+00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
25%,0.0,0.0,0.0,1.442e-03,1.442e-03,0.038,0.038,0.0,0.0,0.0,0.0,1.000,1.000,1.000,1.0

## Generating Features